# **🔫 Weapon Detection AI**

In [ ]:
# Install YOLOv8 (Ultralytics) and Kaggle
!pip install ultralytics kaggle -q
import os
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
# 1. Install YOLOv8 (Ultralytics)
!pip install ultralytics -q
import os
import shutil
import glob
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# 2. Clone the repository
!rm -rf OD-WeaponDetection  # Clean up if exists
!git clone https://github.com/ari-dasci/OD-WeaponDetection.git

print("Repository cloned successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Cloning into 'OD-WeaponDetection'...
remote: Enumerating objects: 50755, done.
remote: Total 50755 (delta 0), reused 0 (delta 0), pack-reused 50755 (from 1)
Receiving objects: 100% (50755/50755), 2.26 GiB | 40.19 MiB/s, done.
Resolving deltas: 100% (10983/10983), done.
Updating files: 100% (63128/63128), done.
Repository cloned successfully.


# Task
Prepare the dataset for YOLOv8 by converting existing annotations into YOLO format, normalizing bounding box coordinates, and splitting the data into training, validation, and test sets, then organizing them within the `/content/yolo_dataset` directory from the `OD-WeaponDetection` repository.

## Prepare Dataset for YOLOv8

### Subtask:
Process raw dataset by converting existing XML annotations to YOLO format with normalized bounding box coordinates, and then organizing images and labels into training, validation, and test sets within the '/content/yolo_dataset' directory.


**Reasoning**:
The first step is to define the `class_map` to assign integer IDs to the object classes, and then set up the necessary directory structure for the YOLO dataset, including train, validation, and test subdirectories for both images and labels.



In [5]:
import os
import shutil
import glob
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from PIL import Image

# 1. Define class_map
class_map = {'pistol': 0, 'knife': 1}
print(f"Class map defined: {class_map}")

# 2. Define output directory and create subdirectories
output_dir = '/content/yolo_dataset'

# Clean up previous runs if any
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(os.path.join(output_dir, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images', 'test'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'test'), exist_ok=True)
print(f"Created YOLO dataset directory structure at: {output_dir}")

Class map defined: {'pistol': 0, 'knife': 1}
Created YOLO dataset directory structure at: /content/yolo_dataset


**Reasoning**:
The next step is to define the `convert_to_yolo` function as described in the instructions. This function will parse XML annotation files, convert bounding box coordinates to the normalized YOLO format, and use the `class_map` to assign integer IDs to classes.



In [6]:
def convert_to_yolo(xml_file_path, img_width, img_height, class_map):
    yolo_annotations = []
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_map:
            print(f"Warning: Class '{class_name}' not in class_map. Skipping object.")
            continue
        class_id = class_map[class_name]

        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Convert to YOLO format (x_center, y_center, width, height) normalized
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_annotations

print("Defined 'convert_to_yolo' function.")

Defined 'convert_to_yolo' function.


**Reasoning**:
The next step is to initialize an empty list `all_image_paths` and then iterate through the `source_folders`. For each folder, the code will find all image and XML annotation file pairs, extract image dimensions, convert XML annotations to YOLO format using the `convert_to_yolo` function, save the YOLO annotations to `.txt` files, and finally populate the `all_image_paths` list with tuples of image and label file paths.



In [7]:
all_image_paths = []

# Source folders containing images and XML annotations
source_folders = [
    '/content/OD-WeaponDetection/Pistol detection',
    '/content/OD-WeaponDetection/Knife_detection'
]

# Create a temporary directory for YOLO labels before splitting
temp_label_dir = os.path.join(output_dir, 'temp_yolo_labels')
os.makedirs(temp_label_dir, exist_ok=True)

for folder in source_folders:
    print(f"Processing folder: {folder}")
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_files.extend(glob.glob(os.path.join(folder, ext)))

    for img_path in image_files:
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        xml_path = os.path.join(folder, base_name + '.xml')

        if os.path.exists(xml_path):
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size

                yolo_annotations = convert_to_yolo(xml_path, img_width, img_height, class_map)

                # Save YOLO annotations to a temporary .txt file
                yolo_label_path = os.path.join(temp_label_dir, base_name + '.txt')
                with open(yolo_label_path, 'w') as f:
                    for line in yolo_annotations:
                        f.write(line + '\n')

                all_image_paths.append((img_path, yolo_label_path))
            except Exception as e:
                print(f"Error processing {img_path} or {xml_path}: {e}")
        else:
            print(f"Warning: XML file not found for {img_path}. Skipping.")

print(f"Total image-annotation pairs processed: {len(all_image_paths)}")

Processing folder: /content/OD-WeaponDetection/Pistol detection
Processing folder: /content/OD-WeaponDetection/Knife_detection
Total image-annotation pairs processed: 0


**Reasoning**:
The previous execution processed 0 image-annotation pairs, indicating an issue with file path detection. The problem likely stems from incorrect paths for images and XML annotations. I need to re-examine the directory structure and correct the `glob.glob` pattern to accurately locate the image and XML files, which are typically within an 'images' subdirectory inside the `source_folders`.



In [8]:
all_image_paths = []

# Source folders containing images and XML annotations
source_folders = [
    '/content/OD-WeaponDetection/Pistol detection',
    '/content/OD-WeaponDetection/Knife_detection'
]

# Create a temporary directory for YOLO labels before splitting
temp_label_dir = os.path.join(output_dir, 'temp_yolo_labels')
os.makedirs(temp_label_dir, exist_ok=True)

for folder in source_folders:
    print(f"Processing folder: {folder}")

    # Images and XMLs are likely in a subdirectory named 'images' within the main folder
    image_subfolder = os.path.join(folder, 'images')

    if not os.path.exists(image_subfolder):
        print(f"Warning: Image subfolder not found at {image_subfolder}. Skipping.")
        continue

    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_files.extend(glob.glob(os.path.join(image_subfolder, ext)))

    for img_path in image_files:
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        # Assuming XMLs are in the same 'images' subfolder as the images
        xml_path = os.path.join(image_subfolder, base_name + '.xml')

        if os.path.exists(xml_path):
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size

                yolo_annotations = convert_to_yolo(xml_path, img_width, img_height, class_map)

                # Save YOLO annotations to a temporary .txt file
                yolo_label_path = os.path.join(temp_label_dir, base_name + '.txt')
                with open(yolo_label_path, 'w') as f:
                    for line in yolo_annotations:
                        f.write(line + '\n')

                all_image_paths.append((img_path, yolo_label_path))
            except Exception as e:
                print(f"Error processing {img_path} or {xml_path}: {e}")
        else:
            print(f"Warning: XML file not found for {img_path}. Skipping.")

print(f"Total image-annotation pairs processed: {len(all_image_paths)}")

Processing folder: /content/OD-WeaponDetection/Pistol detection
Processing folder: /content/OD-WeaponDetection/Knife_detection
Total image-annotation pairs processed: 0


**Reasoning**:
The previous attempts to locate image and XML files within `source_folders` have failed, indicating my assumptions about the directory structure are incorrect. I need to inspect the contents of the `OD-WeaponDetection` repository to identify the correct paths to the image and annotation files. I'll use a shell command to list the files and subdirectories within the `source_folders` to understand the actual layout.



In [9]:
import os

# Inspect the contents of the source folders
for folder in source_folders:
    print(f"Listing contents of {folder}:")
    !ls -F "{folder}"
    print("\n")

# Also inspect a bit deeper into one of the likely candidates if they exist, e.g., 'images' or 'annotations' within the source folder
# Let's try inspecting '/content/OD-WeaponDetection/Pistol detection/Images' and '/content/OD-WeaponDetection/Pistol detection/Annotations'

pistol_images_path = os.path.join(source_folders[0], 'Images')
pistol_annotations_path = os.path.join(source_folders[0], 'Annotations')
knife_images_path = os.path.join(source_folders[1], 'Images')
knife_annotations_path = os.path.join(source_folders[1], 'Annotations')

if os.path.exists(pistol_images_path):
    print(f"Listing contents of {pistol_images_path}:")
    !ls -F "{pistol_images_path}" | head -n 5
    print("(Showing first 5 items)\n")

if os.path.exists(pistol_annotations_path):
    print(f"Listing contents of {pistol_annotations_path}:")
    !ls -F "{pistol_annotations_path}" | head -n 5
    print("(Showing first 5 items)\n")

if os.path.exists(knife_images_path):
    print(f"Listing contents of {knife_images_path}:")
    !ls -F "{knife_images_path}" | head -n 5
    print("(Showing first 5 items)\n")

if os.path.exists(knife_annotations_path):
    print(f"Listing contents of {knife_annotations_path}:")
    !ls -F "{knife_annotations_path}" | head -n 5
    print("(Showing first 5 items)\n")


Listing contents of /content/OD-WeaponDetection/Pistol detection:
Weapons/  xmls/


Listing contents of /content/OD-WeaponDetection/Knife_detection:
annotations/  Images/


Listing contents of /content/OD-WeaponDetection/Knife_detection/Images:
DefenseAndSurvive02.jpg*
DefenseAndSurvive04.jpg*
DefenseAndSurvive05.jpg*
DefenseAndSurvive07.jpg*
DefenseAndSurvive09.jpg*
(Showing first 5 items)



**Reasoning**:
Based on the inspection of the directory structure, the image and XML annotation files are located in different subdirectories for 'Pistol detection' and 'Knife_detection'. I need to adjust the code to correctly identify these paths and process the image-annotation pairs accordingly to populate `all_image_paths`.



In [10]:
all_image_paths = []

# Source folders containing images and XML annotations
source_folders = {
    'pistol': {'base': '/content/OD-WeaponDetection/Pistol detection', 'images': 'Weapons', 'annotations': 'xmls'},
    'knife': {'base': '/content/OD-WeaponDetection/Knife_detection', 'images': 'Images', 'annotations': 'annotations'}
}

# Create a temporary directory for YOLO labels before splitting
temp_label_dir = os.path.join(output_dir, 'temp_yolo_labels')
os.makedirs(temp_label_dir, exist_ok=True)

for weapon_type, paths in source_folders.items():
    print(f"Processing {weapon_type} detection folder: {paths['base']}")

    image_source_path = os.path.join(paths['base'], paths['images'])
    annotation_source_path = os.path.join(paths['base'], paths['annotations'])

    if not os.path.exists(image_source_path):
        print(f"Warning: Image folder not found at {image_source_path}. Skipping.")
        continue
    if not os.path.exists(annotation_source_path):
        print(f"Warning: Annotation folder not found at {annotation_source_path}. Skipping.")
        continue

    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_files.extend(glob.glob(os.path.join(image_source_path, ext)))

    for img_path in image_files:
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        xml_path = os.path.join(annotation_source_path, base_name + '.xml')

        if os.path.exists(xml_path):
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size

                yolo_annotations = convert_to_yolo(xml_path, img_width, img_height, class_map)

                # Save YOLO annotations to a temporary .txt file
                yolo_label_path = os.path.join(temp_label_dir, base_name + '.txt')
                with open(yolo_label_path, 'w') as f:
                    for line in yolo_annotations:
                        f.write(line + '\n')

                all_image_paths.append((img_path, yolo_label_path))
            except Exception as e:
                print(f"Error processing {img_path} or {xml_path}: {e}")
        else:
            print(f"Warning: XML file not found for {img_path}. Skipping.")

print(f"Total image-annotation pairs processed: {len(all_image_paths)}")

Processing pistol detection folder: /content/OD-WeaponDetection/Pistol detection
Processing knife detection folder: /content/OD-WeaponDetection/Knife_detection
Total image-annotation pairs processed: 4948


**Reasoning**:
The next step is to split the `all_image_paths` into training, validation, and test sets using `train_test_split` from `sklearn.model_selection`. This involves two splits to achieve the desired 70/20/10 ratio.



In [11]:
from sklearn.model_selection import train_test_split

# Split into training (70%) and temp (30% for val+test)
train_paths, temp_paths = train_test_split(all_image_paths, test_size=0.3, random_state=42)

# Split temp into validation (20%) and test (10%)
# 0.6666... because 20% of 100% is 2/3 of 30%
val_paths, test_paths = train_test_split(temp_paths, test_size=0.3333333, random_state=42)

print(f"Total images: {len(all_image_paths)}")
print(f"Training images: {len(train_paths)}")
print(f"Validation images: {len(val_paths)}")
print(f"Test images: {len(test_paths)}")

Total images: 4948
Training images: 3463
Validation images: 990
Test images: 495


**Reasoning**:
The final step is to copy the images and their corresponding YOLO label files from the temporary locations to their designated train, validation, and test directories within the `output_dir`.



In [12]:
dataset_splits = {
    'train': train_paths,
    'val': val_paths,
    'test': test_paths
}

for split, paths_list in dataset_splits.items():
    print(f"Copying files for {split} split...")
    for img_src, label_src in paths_list:
        # Determine destination paths
        img_dst_dir = os.path.join(output_dir, 'images', split)
        label_dst_dir = os.path.join(output_dir, 'labels', split)

        # Copy image file
        shutil.copy(img_src, os.path.join(img_dst_dir, os.path.basename(img_src)))

        # Copy label file
        shutil.copy(label_src, os.path.join(label_dst_dir, os.path.basename(label_src)))

print("Dataset preparation complete. Files are organized in '/content/yolo_dataset'.")

# Clean up the temporary label directory
shutil.rmtree(temp_label_dir)
print(f"Cleaned up temporary label directory: {temp_label_dir}")

Copying files for train split...
Copying files for val split...
Copying files for test split...
Dataset preparation complete. Files are organized in '/content/yolo_dataset'.
Cleaned up temporary label directory: /content/yolo_dataset/temp_yolo_labels


## Create YOLOv8 Configuration File

### Subtask:
Generate the 'data.yaml' file, which is crucial for YOLOv8 training. This file will define the paths to your training, validation, and test image directories and list the names of the object classes (e.g., 'pistol', 'knife') that your model will learn to detect.


**Reasoning**:
To complete the subtask, I need to generate the 'data.yaml' file as specified in the instructions. This involves defining the paths for training, validation, and testing images, determining the number of classes, listing class names from the `class_map`, and then writing this information in YAML format to the `data.yaml` file within the `output_dir`.



In [13]:
import os

# 1. Define paths for training, validation, and test image directories
train_path = os.path.join('images', 'train')
val_path = os.path.join('images', 'val')
test_path = os.path.join('images', 'test')

# 2. Define the number of classes (nc)
nc = len(class_map)

# 3. Define the class names (names) in order
# Sort class_map by values (IDs) to ensure correct order
names = [k for k, v in sorted(class_map.items(), key=lambda item: item[1])]

# 4. Create the data.yaml file path
data_yaml_path = os.path.join(output_dir, 'data.yaml')

# 5. Write the defined paths, number of classes, and class names into the data.yaml file
with open(data_yaml_path, 'w') as f:
    f.write(f"path: {output_dir}\n") # Base path for dataset
    f.write(f"train: {train_path}\n")
    f.write(f"val: {val_path}\n")
    f.write(f"test: {test_path}\n")
    f.write(f"\nnc: {nc}\n")
    f.write(f"names: {names}\n")

print(f"'data.yaml' created successfully at {data_yaml_path}:")
with open(data_yaml_path, 'r') as f:
    print(f.read())

'data.yaml' created successfully at /content/yolo_dataset/data.yaml:
path: /content/yolo_dataset
train: images/train
val: images/val
test: images/test

nc: 2
names: ['pistol', 'knife']



## Train YOLOv8 Model

### Subtask:
Initialize a YOLOv8 object detection model and begin the training process. The model will be trained using the prepared dataset and the configurations specified in the 'data.yaml' file. This step will also include setting training parameters such as the number of epochs and image size.


**Reasoning**:
The subtask requires initializing a YOLOv8 model and starting the training process with specified parameters, using the previously created 'data.yaml' file.



In [14]:
from ultralytics import YOLO

# 1. Instantiate a YOLOv8 model (e.g., yolov8n.pt)
model = YOLO('yolov8n.pt')
print("YOLOv8n model loaded.")

# 2. Start the training process
# 3. Pass the path to the data.yaml file
# 4. Set the number of training epochs to 50
# 5. Set the image size for training to 640
results = model.train(data=data_yaml_path, epochs=50, imgsz=640)

print("Model training initiated.")

YOLOv8n model loaded.
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspectiv

In [16]:
from google.colab import files
files.download(f'/content/runs/detect/train/weights/best.pt')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>